In [35]:
import time
import threading
import matplotlib.pyplot as plt
%matplotlib inline
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [119]:
class Philosopher(threading.Thread):
    def __init__(self, name, both_forks):
        """
        both_forks = [1, 2]
        """
        super().__init__()
        self.name = name
        self.both_forks = both_forks #食事に必要なフォーク番号
        self.hold_forks = [] #現時点で持っているフォーク番号
        self.state = 0 
        """
        self.state: 現在の状況
        0: hold noting(hungry)   
        1:pickup1, 
        2:pickup2(eating), 
        3:puts1, 
        4:puts2(thinking)
        """
        
    def do_fork(self, Forks):
        if self.state == 0:
            #######はじめにどちらのフォークを持つか##########
#             wait_fork = random.choice(self.both_forks) #ランダム
            wait_fork = self.both_forks[0] #左から取る
#             wait_fork = self.both_forks[1] #右から取る
            ###################################
        
        
            get_forks(wait_fork, Forks, self.hold_forks, self.name)
            self.state = 1
            
        elif self.state == 1:
            wait_fork = list(set(self.both_forks) - set(self.hold_forks))[0]

            get_forks(wait_fork, Forks, self.hold_forks, self.name)
            
            self.state = 2
            
        elif self.state == 2:
            #######始めにどちらのフォークを置くか##########
#             put_fork = random.choice(self.hold_forks) #ランダム
#             put_fork = self.both_forks[0] #左から置く
            put_fork = self.both_forks[1] #右から置く
            ###################################
            
            put_forks(put_fork, Forks, self.hold_forks, self.name)
            self.state = 3
            event.set()
            
        elif self.state == 3: #食事終了
            put_fork = self.hold_forks[0]
            put_forks(put_fork, Forks, self.hold_forks, self.name)
            self.state = 4
            event.set()
        
    def action(self, Forks):
        for i in range(30):
#             print("trial : {}".format(i))

            t = random.random() * 2
            if self.state == 0:
                print("{} is hungry for {:.3f}sec.".format(self.name, t))
                wait_time(t)
                self.do_fork(Forks)

            elif self.state == 2:
                print("{} is eating for {:.3f}sec.".format(self.name, t))
                wait_time(t)
                self.do_fork(Forks)

            elif self.state == 1 or self.state == 3:
                self.do_fork(Forks)
                
            elif self.state == 4:
                print("{} is thinking for {:.3f}sec.".format(self.name, t))
                wait_time(t)
                self.state = 0
            

In [120]:
def wait_time(t):
    time.sleep(t)

In [121]:
def put_forks(fork_num, Forks_class, hold_forks, name):
    Forks_class.change_state(fork_num)
    print("{} puts {}.".format(name,fork_num))
#     print("hold_forks", hold_forks)
    hold_forks.remove(fork_num)
    event.set()

In [122]:
def get_forks(fork_num, Forks_class, hold_forks, name):
    start_time = time.time()
    while not Forks_class.get_fork_state(fork_num):
        event.wait()
    end_time = time.time()
        
    Forks_class.change_state(fork_num) #使用中に変更
    hold_forks.append(fork_num)
    need_time = end_time - start_time
    print("{} picks up {}. It takes {:.3f}sec".format(name, fork_num, need_time*1000))

In [123]:
class Forks:
    def __init__(self):
        self.fork_num_dic = {1: True, 2:True, 3:True, 4:True}
        
    def get_fork_state(self, fork_num):
        return self.fork_num_dic[fork_num] #True：空き、False:使用中
    
    def change_state(self, fork_num):
        if self.fork_num_dic[fork_num]: #TrueならFlaseへ
            self.fork_num_dic[fork_num] = False
        else:
            self.fork_num_dic[fork_num] = True
        

In [124]:

fork = Forks()
phi_a = Philosopher("a", [1, 2])
phi_b = Philosopher("b", [2, 1])

In [125]:
event = threading.Event()

In [126]:
thread_a = threading.Thread(target= phi_a.action, args=(fork, ))
thread_b = threading.Thread(target= phi_b.action, args=(fork, ))

In [127]:
thread_a.start()
thread_b.start()

a is hungry for 0.952sec.
b is hungry for 0.166sec.
b picks up 2. It takes 0.005sec
b picks up 1. It takes 0.001sec
b is eating for 1.621sec.
b puts 1.
b puts 2.
b is thinking for 1.015sec.
a picks up 1. It takes 839.326sec
a picks up 2. It takes 0.002sec
a is eating for 0.080sec.
a puts 2.
a puts 1.
a is thinking for 0.591sec.
a is hungry for 1.109sec.
b is hungry for 0.867sec.
a picks up 1. It takes 0.009sec
a picks up 2. It takes 0.001sec
a is eating for 0.209sec.
a puts 2.b picks up 2. It takes 120.761sec
a puts 1.
a is thinking for 0.678sec.

b picks up 1. It takes 0.000sec
b is eating for 0.008sec.
b puts 1.
b puts 2.
b is thinking for 1.983sec.
a is hungry for 0.708sec.
a picks up 1. It takes 0.007sec
a picks up 2. It takes 0.001sec
a is eating for 0.472sec.
a puts 2.
a puts 1.
a is thinking for 1.493sec.
b is hungry for 1.788sec.
a is hungry for 0.057sec.
a picks up 1. It takes 0.009sec
a picks up 2. It takes 0.001sec
a is eating for 0.256sec.
a puts 2.
a puts 1.
a is thinking 

In [10]:

fork = Forks()
phi_a = Philosopher("a", [1, 2])
phi_b = Philosopher("b", [2, 3])
phi_c = Philosopher("c", [3, 4])
phi_d = Philosopher("d", [4, 1])

In [11]:
event = threading.Event()

In [12]:
thread_a = threading.Thread(target= phi_a.action, args=(fork, ))
thread_b = threading.Thread(target= phi_b.action, args=(fork, ))
thread_c = threading.Thread(target= phi_c.action, args=(fork, ))
thread_d = threading.Thread(target= phi_d.action, args=(fork, ))

In [13]:
thread_a.start()
thread_b.start()
thread_c.start()
thread_d.start()

a is hungry.
b is hungry.
c is hungry.
d is hungry.
b picks up 2.
b picks up 3.
b is eating.
d picks up 4.
d picks up 1.
d is eating.
b puts 3.
b puts 2.
b is thinking.
a picks up 2.
d puts 4.c picks up 4.
d puts 1.
d is thinking.
a picks up 1.
c picks up 3.
c is eating.

a is eating.
c puts 4.
c puts 3.
c is thinking.
b is hungry.
d is hungry.
a puts 1.d picks up 1.

d picks up 4.
d is eating.
a puts 2.
a is thinking.
b picks up 2.
b picks up 3.
b is eating.
d puts 4.
d puts 1.
d is thinking.
c is hungry.
a is hungry.
b puts 3.
b puts 2.
b is thinking.
a picks up 2.
a picks up 1.
a is eating.
c picks up 3.
c picks up 4.
c is eating.
c puts 3.
c puts 4.
c is thinking.
a puts 2.
a puts 1.
a is thinking.
